# 爬取ettoday (最好爬)

In [1]:
import time
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
import requests
from lxml import etree
import os
import threading
import queue
import pymongo
from tqdm import trange,tqdm
# chrome_options = Options()
# chrome_options.add_argument('--headless')
# driver = webdriver.Chrome(options=chrome_options)

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["semester"]
mycol = mydb["ettoday"]

In [2]:
month_j = {
    "8":31,"9":30,"10":31,"11":30,"12":31
}

In [3]:
url = "https://www.ettoday.net/news/news-list-2019-{}-{}-0.htm"
def getDateNews(mon,date,D):
    links = []
    tags = []
    response = requests.get(url.format(mon,date))
    html = etree.HTML(response.content)
    links = html.xpath("//div[@class='part_list_2']/h3/a/@href")
    links = ["https://www.ettoday.net"+link for link in links]
    tags = html.xpath("//div[@class='part_list_2']/h3/em/text()")
    data = {"offset": 1,"tPage": 3,"tFile": D+".xml","tOt": 0,"tSi": 100,"tAr":0}
    for i in range(1,69):
        data["offset"] = i
        response = requests.post("https://www.ettoday.net/show_roll.php",data=data)
        html = etree.HTML(response.text)
        h3 = html.xpath("//h3")
        for k in h3:
            links.append("https://www.ettoday.net"+k.xpath(".//a/@href")[0])
            tags.append(k.xpath(".//text()")[1])
    return tags,links

In [4]:
### 用selenium寫，效果不好，太慢了
url = "https://www.ettoday.net/news/news-list-2019-{}-{}-0.htm"
def getDateNews(mon,date,D):
    driver.get(url.format(mon,date))
    time.sleep(2)
    for i in range(80):
        if(i==10 or i== 20):
            driver.execute_script("window.scrollTo(document.body.scrollHeight,0)")
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(0.2)
    news = driver.find_elements_by_xpath("//div[@class='part_list_2']/h3/a")
    links = [link.get_attribute("href") for link in news]
    tags_ = driver.find_elements_by_xpath("//div[@class='part_list_2']/h3/em")
    tags = [tag.text for tag in tags_]
    return tags,links

In [4]:
def getnews(link,D,tag):
    try: 
        response = requests.get(link)
        tag = tag
        html = etree.HTML(response.content,parser=etree.HTMLParser(encoding='utf-8'))
        link = link
        title = html.xpath("//header/h1/text()")
        title = title[0].strip()
        contents = html.xpath("//div[@class='story']/p/text()")
        content = "\n".join([content.strip()for content in contents])
        img_link = html.xpath("//div[@class='story']//img/@src")[0]
        pic = requests.get("https:"+img_link)
        link_id = link.split("/")[-1].split(".")[0]
        time = html.xpath("//time/@datetime")[0]
        time = time.replace("+08:00","").replace("T","_").replace(":","-")
        img_path = "./pic/"+D+"/"+time+"_"+link_id+".png"
        open(img_path,"wb").write(pic.content)
        insert_dict = {"title":title,"content":content,"time":time,"tag":tag,"link":link,"img_path":img_path}
        mycol.update_one({"link":insert_dict["link"]},{"$set":insert_dict},upsert=True)
    except:
        pass

In [5]:
def crawlDateNews(tags,links,D):
    threads = []
    for i,link in enumerate(links):
        date = link.split('/')[-2]        
        if(date != D or mycol.find_one({"link":link}) != None):
            continue
        threads.append(threading.Thread(target=getnews,args=(link,D,tags[i])))

    ## 成功，多執行續!!!!
    q = queue.Queue(16)
    while(len(threads)):
        if(q.qsize() >= 16):
            thread = q.get()
            thread.start()
            continue
        else:
            q.put(threads.pop())

    while(q.qsize()>0):
        thread = q.get()
        thread.start()
        thread.join()

In [6]:
m = time.strftime("%m",time.localtime(time.time()))
d = time.strftime("%d",time.localtime(time.time()))
mo = int(m)
da = int(d)
for j in range(8,mo+1):
    mon = j
    date = month_j[str(mon)]
    if(j==mo):
        date = da
    m = "0"+str(mon) if mon <10 else mon
    for i in tqdm(range(1,date+1),desc="Month "+str(m)):
        d = "0"+str(i) if i <10 else i
        D = "2019"+str(m)+str(d)
        directory = "./pic/"+D
        if not os.path.exists(directory):
            os.makedirs(directory)
        tags,links = getDateNews(str(mon),str(i),D)
        crawlDateNews(tags,links,D)

Month 12: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [01:28<00:00, 44.46s/it]
